In [1]:
#### Basic Setting

import  load_dotenv
load_dotenv.load_dotenv("../../All_LLM_tutorial/.env")

True

### Auto parsing

In [6]:
from llama_index.core.node_parser import SimpleFileNodeParser
from llama_index.readers.file import FlatReader
from pathlib import Path

documents = FlatReader().load_data(Path("./data/paul_graham_essay.txt"))

parser = SimpleFileNodeParser()
md_nodes = parser.get_nodes_from_documents(documents)

### Other parsers

In [30]:
# temp_data.txt 파일을 읽어서 내용을 가져옵니다.
with open("./data/temp_data.txt", "r", encoding="utf-8") as f:
    temp_data_text = f.read()

print("temp_data.txt 내용 일부 미리보기:")
print(temp_data_text[300:310])  # 앞부분 300자만 출력

print(temp_data_text[600:610])


temp_data.txt 내용 일부 미리보기:
--- 구분자 --
-- 구분자 ---


In [34]:
documents = FlatReader().load_data(Path("./data/temp_data.txt"))

In [35]:
#랭체인 없으면 안돌아가서 인증 두배!
from langchain.text_splitter import RecursiveCharacterTextSplitter
from llama_index.core.node_parser import LangchainNodeParser

parser = LangchainNodeParser(RecursiveCharacterTextSplitter())
nodes = parser.get_nodes_from_documents(documents)

In [39]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(
    chunk_size=300,
    chunk_overlap=20,
)
nodes = splitter.get_nodes_from_documents(documents)

In [46]:
print(len(nodes[0].get_text()))
print(len(nodes[1].get_text()))
print(len(nodes[2].get_text()))

252
247
273


## Semantic Chunking

In [50]:
from llama_index.core.node_parser import SemanticSplitterNodeParser
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model = OpenAIEmbedding()
splitter = SemanticSplitterNodeParser(
    buffer_size=1, breakpoint_percentile_threshold=95, embed_model=embed_model
)

In [51]:
splitter.get_nodes_from_documents(documents)

[TextNode(id_='a7c7475e-da69-4ca4-8efb-25ced1d7750f', embedding=None, metadata={'filename': 'temp_data.txt', 'extension': '.txt'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='ff4c4bee-a03f-434d-b39e-b61f014deef7', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'filename': 'temp_data.txt', 'extension': '.txt'}, hash='3eb381fbbc54a4d0848988c23a893ebb2b8203de88a04290d79496212e92f46b'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='381c8795-e625-4eed-a63a-3173c6fb68ec', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='b8e4dbbaf6103deda9334c2e25e1f14602d1209d7bcbfe7d9ab0a743727543a8')}, metadata_template='{key}: {value}', metadata_separator='\n', text='오늘 오전 서울 중구에서는 대규모 인공지능 컨퍼런스가 열려 국내외 AI 전문가들이 한자리에 모였다. 이번 컨퍼런스의 핵심 주제는 생성형 인공지능의 산업 적용 사례와 미래 전망이었으며, 기업·학계·스타트업 관계자들이 기술적 과제와 성공 전략에 대해 심도 있게 논의했다. 기조연설에서는 대규모 언어 모델의 최신 연구 동향과 윤리적 고려 사항이 소개되었고, 패널 토론에서는 실제 도입 과정에서 발생한 데이터 품질 관리 문제와 리스크

### Token chunking

In [ ]:
from llama_index.core.node_parser import TokenTextSplitter

splitter = TokenTextSplitter(
    chunk_size=1024,
    chunk_overlap=20,
    separator=" ",
)
nodes = splitter.get_nodes_from_documents(documents)

### 계층적, 아마도 Multi retriever

In [52]:
from llama_index.core.node_parser import HierarchicalNodeParser

node_parser = HierarchicalNodeParser.from_defaults(
    chunk_sizes=[100, 200, 300]
)

### 구분자 test

In [84]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(
    chunk_size=300,
    chunk_overlap=0,
    # chunk_overlap=20,
    # separator="구분자",
    paragraph_separator="구분자",
    secondary_chunking_regex=None
)
nodes = splitter.get_nodes_from_documents(documents)

In [87]:
splitter._split_fns

[<function llama_index.core.node_parser.text.utils.split_by_sep.<locals>.<lambda>(text)>,
 <function llama_index.core.node_parser.text.utils.split_by_sentence_tokenizer.<locals>.<lambda>(text)>]

In [85]:
[i.get_text() for i in nodes]

['오늘 오전 서울 중구에서는 대규모 인공지능 컨퍼런스가 열려 국내외 AI 전문가들이 한자리에 모였다. 이번 컨퍼런스의 핵심 주제는 생성형 인공지능의 산업 적용 사례와 미래 전망이었으며, 기업·학계·스타트업 관계자들이 기술적 과제와 성공 전략에 대해 심도 있게 논의했다. 기조연설에서는 대규모 언어 모델의 최신 연구 동향과 윤리적 고려 사항이 소개되었고, 패널 토론에서는 실제 도입 과정에서 발생한 데이터 품질 관리 문제와 리스크 대응 방안이 공유되었다.',
 '참석자들은 혁신 가속화를 위한 협업 생태계 구축의 필요성에 공감하였다. 행사 열기증\n---',
 '구분자 ---\n오후 세션에서는 오픈소스 생태계에서의 기여 사례와 멀티모달 학습 프레임워크의 성능 최적화 기법이 집중 조명되었다. 특히 한국어 자연어 처리 문제를 해결하기 위한 공개 데이터셋 구축 경험이 소개되어 뜨거운 반응을 얻었다. 연사를 맡은 연구진은 파트너십을 기반으로 한 데이터 품질 개선 프로세스와 모델 재학습 주기에 대해 상세히 설명하였다. 또한 실제 서비스 배포 과정에서의 지연 시간 감소를 위해 ONNX 및 TensorRT 최적화가 어떻게 적용되었는지 데모를 통해 시연해 참석자들의 이해를 도왔다. 성능 지표까지\n---',
 '구분자 ---\n마지막 세션에서는 AI 윤리와 규제 대응 전략이 다뤄졌다. 발표자는 개인정보보호법 개정안과 국제 표준 동향을 소개하며, 모델 학습 단계에서부터 프라이버시 보호 설계를 적용하는 방법론을 설명했다. 이어진 사례 발표에서는 기업이 투명성 보고서를 통해 이해관계자와 신뢰를 구축한 경험을 공유하였다. 참석자들은 정책 변화에 선제적으로 대응하고자 자사 내부 가이드라인을 정비할 필요성을 논의했다. 행사 종료 후 네트워킹 시간에는 투자사와 개발자들이 협업 가능성을 모색하며 명함을 교환하였다.',
 '스타트업 실무자 고민을 들을 수 있어 의미가 컸다 매우\n--- 마지막 구분자 ---']